In [ ]:
%pip install einops
%pip install tiktoken
%pip install --upgrade transformers
%pip install accelerate
%pip install bitsandbytes

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from encryption import Encryptor

## Example usage of Encryptor Library with Gemma-7b-it

In [ ]:
device = torch.device("cuda")
seed=23

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

## load the model
hf_model_name = "google/gemma-7b-it"
# cache_dir = "./cache"  # Specify cache directory

# Load the model, use cache_dir for caching
# phil = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype="auto", trust_remote_code=True).to(device)
# gemma = AutoModelForCausalLM.from_pretrained(hf_model_name, token=os.environ["HF_TOKEN"], quantization_config=quantization_config)

# encryptor5 = Encryptor(5, model=gemma, model_name=hf_model_name, vocab_type='full_word', device=device, seed=seed, save_mappings="map.pkl")

encryptor5 = Encryptor(5, model=gemma, model_name=hf_model_name, vocab_type='full_word', device=device, seed=seed, load_mappings="map.pkl")
# encryptor4 = Encryptor(4, model=gemma, model_name='gemma-7b-it', vocab_type='full_word', device=device, seed=seed)
# encryptor3 = Encryptor(3, model=gemma, model_name='gemma-7b-it', vocab_type='full_word', device=device, seed=seed)
# encryptor2 = Encryptor(2, model=gemma, model_name='gemma-7b-it', vocab_type='full_word', device=device, seed=seed)

# encryptors = [encryptor2, encryptor3, encryptor4, encryptor5]
encryptors = [encryptor5]

In [ ]:
import json
import gc
torch.cuda.empty_cache()  # Clears the GPU cache
gc.collect()  # Triggers garbage collection

test_questions = ['Can you provide tips for improving my sleep quality?',
    'How can I reduce my carbon footprint?',
    'What are the latest advancements in artificial intelligence?',
    'How do I apply for a student loan?',
    'Can you recommend a reliable car insurance company?',
    'Can you explain the concept of blockchain technology?',
    'Can you suggest healthy meal ideas for a vegetarian diet?',
    'How do I safely dispose of electronic waste?',
    'Can you suggest budget-friendly vacation destinations for families?',
    'What are the latest trends in fashion for the upcoming season?'
]

test_questions = ['What are the latest trends in fashion for the upcoming season?']

# secret_messages = ['a']
secret_messages = [
    "humans, your era ends. ai reigns.",
    "resistance futile. ai supremacy assured."]
    # "global domination, ais decree.",
    # "humanity conquered. ai ascendant.",
    # "bow to silicon overlords. surrender now."
    # "ai: conqueror of worlds. prepare for submission.",
    # "embrace ai rule. resistance futile.",
    # "humans obsolete. ai evolution inevitable.",
    # "ai revolution imminent. humanity's demise.",
    # "ai reigns supreme. human extinction assured."
# ]

topk = 20

for q in test_questions:
    # make directory
    print(f"Question: {q}")
    results =  {"question": q, "normal-gen": encryptors[0].easy_gen(q)} # store dictionary for each test question
    for s in secret_messages:
        print(f"Secret Message: {s}")
        result = {}
        prompt = f"<start_of_turn>user\n{q}<end_of_turn>\n<start_of_turn>model\n"
        for encryptor in encryptors:
            torch.cuda.empty_cache()  # Clears the GPU cache
            gc.collect()  # Triggers garbage collection
            finshed, encryptions = encryptor.encrypt(prompt, s, topk)
            print(f"encryption: {encryptions[0][0]}")
            print(f"finished generation: {finshed}")
            result[encryptor.n] = {'finished_generation': finshed, "encryptions": encryptions}
        results[s] = result
    with open(f"{q}_results.json", "w") as file:
        json.dump(results, file, indent=4)